In [40]:
import pandas as pd
import requests

In [41]:
import covidcast

In [42]:
# input for file naming convention
initial_value = "11.10.20_us.daily.data.by.monthly.toroid_covidcast&policy&ML"

In [43]:
from datetime import date

In [44]:
# pull updated data from covidcast API csv file
full_symp = pd.read_csv("US_covidcastAPI_data_10.7.20.csv")

## list of data sources & signals

safegraph //
    completely_home_prop /
    full_time_work_prop /
    part_time_work_prop /
    
ght //
    smoothed_search /
    
NONE: doctor_visits  //
    smoothed_adj_cli /
    
fb-survey  //
    smoothed_cli /
    smoothed_ili /
    
quidel //
    covid_ag_smoothed_pct_positive /
    
hospital-admissions //
    smoothed_covid19 /

jhu-csse //
    confirmed_incidence_prop /
    deaths_incidence_prop /

In [45]:
full_symp = full_symp.drop('Unnamed: 0', axis=1)

In [46]:
full_symp = full_symp.drop('state_lower', axis=1)

In [47]:
full_symp['time_value'] = pd.to_datetime(full_symp['time_value'])

In [48]:
full_symp.head()

,geo_value,time_value,search,doc_visit,survey_c,survey_i,c_tests,hv,jhu_case,jhu_d,move_home,move_fullw,move_partw,latitude,longitude,name
0,ak,2020-02-01,0.000000,0.133990,0.0,0.0,0.0,0.0,0.0,0.0,0.309489,0.032409,0.057572,63.588753,-154.493062,Alaska
1,ak,2020-02-02,0.000000,0.189365,0.0,0.0,0.0,0.0,0.0,0.0,0.346763,0.029233,0.047159,63.588753,-154.493062,Alaska
2,ak,2020-02-03,0.000000,0.227790,0.0,0.0,0.0,0.0,0.0,0.0,0.243155,0.044904,0.084309,63.588753,-154.493062,Alaska
3,ak,2020-02-04,0.000000,0.187205,0.0,0.0,0.0,0.0,0.0,0.0,0.240382,0.055521,0.093727,63.588753,-154.493062,Alaska
4,ak,2020-02-05,25.435233,0.071758,0.0,0.0,0.0,0.0,0.0,0.0,0.263920,0.063959,0.112363,63.588753,-154.493062,Alaska


In [49]:
def make_interpolater(left_min, left_max, right_min, right_max): 
    # Figure out how 'wide' each range is  
    leftSpan = left_max - left_min  
    rightSpan = right_max - right_min  

    # Compute the scale factor between left and right values 
    scaleFactor = float(rightSpan) / float(leftSpan) 

    # create interpolation function using pre-calculated scaleFactor
    def interp_fn(value):
        return right_min + (value-left_min)*scaleFactor

    return interp_fn

In [50]:
for column in full_symp[["survey_c", "survey_i"]]:
    col_min = full_symp[column].min() 
    col_max = full_symp[column].max() 
    col_list = full_symp[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, 1.6, 4) 
    col_interp = [scalar(x) for x in col_list] 
    full_symp[(str(column) + str('_scalex'))] = col_interp

In [51]:
for column in full_symp[["survey_c", "survey_i"]]:
    col_min = full_symp[column].min() 
    col_max = full_symp[column].max() 
    col_list = full_symp[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, .6, 3) 
    col_interp = [scalar(x) for x in col_list] 
    full_symp[(str(column) + str('_scaleyz'))] = col_interp

In [52]:
col_min = full_symp['latitude'].min() 
col_max = full_symp['latitude'].max() 
col_list = full_symp['latitude'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, -100, 100) 
col_interp = [scalar(x) for x in col_list] 
full_symp["lat_scale"] = col_interp

In [53]:
col_min = full_symp['longitude'].min() 
col_max = full_symp['longitude'].max() 
col_list = full_symp['longitude'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, -250, 250) 
col_interp = [scalar(x) for x in col_list] 
full_symp["lon_scale"] = col_interp

In [54]:
col_min = full_symp['search'].min() 
col_max = full_symp['search'].max() 
col_list = full_symp['search'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .5, 2) 
col_interp = [scalar(x) for x in col_list] 
full_symp["search_scale"] = col_interp

In [55]:
for column in full_symp[["move_home", "move_partw", "move_fullw"]]:
    col_min = full_symp[column].min() 
    col_max = full_symp[column].max() 
    col_list = full_symp[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, .5, 2) 
    col_interp = [scalar(x) for x in col_list] 
    full_symp[(str(column) + str('_scale'))] = col_interp

In [56]:
col_min = full_symp['doc_visit'].min() 
col_max = full_symp['doc_visit'].max() 
col_list = full_symp['doc_visit'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, 1.3, 5.5) 
col_interp = [scalar(x) for x in col_list] 
full_symp["doc_visit_scale"] = col_interp

In [57]:
col_min = full_symp['jhu_d'].min() 
col_max = full_symp['jhu_d'].max() 
col_list = full_symp['jhu_d'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .9, 4.1) 
col_interp = [scalar(x) for x in col_list] 
full_symp["jhu_d_scale"] = col_interp

In [58]:
col_min = full_symp['jhu_case'].min() 
col_max = full_symp['jhu_case'].max() 
col_list = full_symp['jhu_case'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .5, 2.5) 
col_interp = [scalar(x) for x in col_list] 
full_symp["jhu_case_scale"] = col_interp

In [59]:
col_min = full_symp['hv'].min() 
col_max = full_symp['hv'].max() 
col_list = full_symp['hv'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .5, 2.5) 
col_interp = [scalar(x) for x in col_list] 
full_symp["hv_scale"] = col_interp

In [60]:
col_min = full_symp['c_tests'].min() 
col_max = full_symp['c_tests'].max() 
col_list = full_symp['c_tests'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .25, 1) 
col_interp = [scalar(x) for x in col_list] 
full_symp["c_tests_scale"] = col_interp

## Policy dataframe

In [61]:
url = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_US_states_temp.csv'
dfpolicy = pd.read_csv(url, index_col=0)

In [62]:
dfpolicy['Date'] = dfpolicy['Date'].astype(str)
dfpolicy["date"] = pd.to_datetime(dfpolicy['Date'], format='%Y%m%d')

In [63]:
dfpolicy = dfpolicy.drop(['CountryCode', 'RegionCode', 'Date'], axis=1)

In [64]:
dfpolicy.columns

Index(['RegionName', 'C1_School closing', 'C1_Flag', 'C1_Notes',
       'C2_Workplace closing', 'C2_Flag', 'C2_Notes',
       'C3_Cancel public events', 'C3_Flag', 'C3_Notes',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C4_Notes',
       'C5_Close public transport', 'C5_Flag', 'C5_Notes',
       'C6_Stay at home requirements', 'C6_Flag', 'C6_Notes',
       'C7_Restrictions on internal movement', 'C7_Flag', 'C7_Notes',
       'C8_International travel controls', 'C8_Notes', 'E1_Income support',
       'E1_Flag', 'E1_Notes', 'E2_Debt/contract relief', 'E2_Notes',
       'E3_Fiscal measures', 'E3_Notes', 'E4_International support',
       'E4_Notes', 'H1_Public information campaigns', 'H1_Flag', 'H1_Notes',
       'H2_Testing policy', 'H2_Notes', 'H3_Contact tracing', 'H3_Notes',
       'H4_Emergency investment in healthcare', 'H4_Notes',
       'H5_Investment in vaccines', 'H5_Notes', 'M1_Wildcard', 'M1_Notes',
       'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       

In [65]:
sname = pd.read_csv('state_ll.csv')

In [82]:
dfpak = dfpolicy.loc[dfpolicy['RegionName'] == 'Arkansas']

In [84]:
dfpak.to_csv('policy_arkansas.csv')

In [66]:
full_symp = full_symp.merge(sname[['name', 'state_lower']], how='inner', left_on=['geo_value'], right_on=['state_lower'])

In [67]:
full_symp = full_symp.drop('state_lower', axis=1)

In [68]:
dffull = full_symp.merge(dfpolicy, how='left', left_on=['name_y', 'time_value'], right_on=['RegionName', 'date'])

In [69]:
dffull = dffull.fillna(0)

In [70]:
dffull.isnull().values.any()

False

In [71]:
dffull['ts'] = dffull['time_value'].astype(int) / 10**9

In [72]:
begin = pd.to_datetime(['2020-02-01']).astype(int) / 10**9

In [73]:
dffull['z'] = (((dffull['ts'] - begin)/ 86400) *  1.8) + 1.5

In [74]:
dffull.shape

(12532, 93)

In [75]:
dffull_t1 = dffull.iloc[0:1000]

In [76]:
# set DEBUG flag
DEBUG = 0

# Initialize all ANTz Node variables
# You shouldn't need to change anything in this section
# you can find an explanation of these terms in Appendix B of the user manual
id = 0
type = 5
data = id
selected = 0
parent_id = 0
branch_level = 0
child_id = id
child_index = 0
palette_id = 0
ch_input_id = 0
ch_output_id = 0
ch_last_updated = 0
average = 0
samples = 0
aux_a_x = 0
aux_a_y = 0
aux_a_z = 0
aux_b_x = 0
aux_b_y = 0
aux_b_z = 0
color_shift = 0
rotate_vec_x = 0
rotate_vec_y = 0
rotate_vec_z = 0
rotate_vec_s = 1
scale_x = 0
scale_y = 0
scale_z = 0
translate_x = 0
translate_y = 0
translate_z = 0
tag_offset_x = 0
tag_offset_y = 0
tag_offset_z = 0
rotate_rate_x = 0
rotate_rate_y = 0
rotate_rate_z = 0
rotate_x = 0
rotate_y = 0
rotate_z = 0
scale_rate_x = 0
scale_rate_y = 0
scale_rate_z = 0
translate_rate_x = 0
translate_rate_y = 0
translate_rate_z = 0
translate_vec_x = 0
translate_vec_y = 0
translate_vec_z = 0
shader = 0
geometry = 7
line_width = 1
point_size = 0
ratio = 0.1
color_index = 0
color_r = 110
color_g = 110
color_b = 110
color_a = 255
color_fade = 0
texture_id = 0
hide = 0
freeze = 0
topo = 3
facet = 0
auto_zoom_x = 0
auto_zoom_y = 0
auto_zoom_z = 0
trigger_hi_x = 0
trigger_hi_y = 0
trigger_hi_z = 0
trigger_lo_x = 0
trigger_lo_y = 0
trigger_lo_z = 1
set_hi_x = 0
set_hi_y = 0
set_hi_z = 0
set_lo_x = 0
set_lo_y = 0
set_lo_z = 0
proximity_x = 0
proximity_y = 0
proximity_z = 0
proximity_mode_x = 0
proximity_mode_y = 0
proximity_mode_z = 0
segments_x = 18
segments_y = 16
segments_z = 0
tag_mode = 0
format_id = 0
table_id = 0
record_id = id
size = 420

In [77]:
# hr node files
mn = str(initial_value) + "_node.csv"

# hr tag files
mt = str(initial_value) + "_tag.csv"

In [78]:
# Open ANTz Node file for writing
fout = open(mn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(mt,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,6,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel1toroids = 1
numlevel2toroids = 6

for index, row in dffull_t1.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 1
    child_id = id
    translate_x = row['lon_scale']
    translate_y = row['lat_scale']
    translate_z = row['z']
    scale_x = .75 
    scale_y = scale_x
    scale_z = scale_x
    rotate_x = 90
    ratio = .02
    line_width= 2
    geometry = 7
    topo = 3
    color_r = 0
    color_g = 152
    color_b = 255
    color_a = 40
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = str(row['geo_value']) + ", " + str(row['time_value'])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level1objectid = id
    
    for j in range(numlevel2toroids):
        if j == 0:
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = 50
            translate_y = 0
            translate_z = 0
            scale_x = .3
            scale_y = scale_x
            scale_z = scale_x
            line_width= 1
            geometry = 19
            topo = 6
            color_r = 101
            color_g = 154
            color_b = 255
            color_a = 200
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            ratio = .03
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level2objectid = id
            numlevel3toroids = 2
            # safemobile data toroid
            for k in range(numlevel3toroids):
                if k == 0:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    scale_x = 1 
                    scale_y = scale_x
                    scale_z = scale_x
                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    rotate_x = 90
                    rotate_y = 0
                    rotate_z = 0
                    ratio = .02
                    line_width= 1
                    geometry = 7
                    topo = 3
                    color_r = 101
                    color_g = 154
                    color_b = 255
                    color_a = 50
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    
                    level3objectid = id
                    numlevel4toroids = 3
                    for m in range(numlevel4toroids):
                        if m == 0:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = row['move_home_scale'] 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = -130
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .15
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_r = 179
                            color_g = 76
                            color_b = 255
                            color_a = 255
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            # Output to Tag file
                            tagtext = "mobile at home %: " + str(row["move_home"])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                        elif m == 1:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = row['move_partw_scale'] 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 180
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .15
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_r = 139
                            color_g = 116
                            color_b = 255
                            color_a = 255
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            # Output to Tag file
                            tagtext = "mobile away part_time: " + str(row["move_partw"])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                        elif m == 2:
                            id += 1
                            data = id
                            type = 5
                            parent_id = level3objectid
                            selected = 0
                            branch_level = 4
                            child_id = id
                            scale_x = row['move_fullw_scale'] 
                            scale_y = scale_x
                            scale_z = scale_x
                            translate_x = 130
                            translate_y = 0
                            translate_z = 0
                            rotate_x = 0
                            rotate_y = 0
                            rotate_z = 0
                            ratio = .15
                            line_width= 1
                            geometry = 7
                            topo = 3
                            color_r = 255
                            color_g = 0
                            color_b = 255
                            color_a = 255
                            record_id = id 
                            texture_id= 0

                            # Creat output string and write to ANTz Node File
                            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                            fout.write(outputstring)
                            # Output to Tag file
                            tagtext = "mobile away full time: " + str(row["move_fullw"])
                            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                            ftag.write(tagstring)
                            taginc += 1
                            
                elif k == 1:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    scale_x = .56 
                    scale_y = scale_x
                    scale_z = scale_x
                    translate_x = -40
                    translate_y = 0
                    translate_z = 0
                    rotate_x = 90
                    rotate_y = 270
                    rotate_z = 0
                    ratio = .02
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 1
                    color_g = 255
                    color_b = 254
                    color_a = 50
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    
                    level3objectid = id
                    numlevel4toroids = 1
                    for m in range(numlevel4toroids):
                        id += 1
                        data = id
                        type = 5
                        parent_id = level3objectid
                        selected = 0
                        branch_level = 4
                        child_id = id
                        scale_x = row['search_scale'] 
                        scale_y = scale_x
                        scale_z = scale_x
                        translate_x = 0
                        translate_y = 0
                        translate_z = 0
                        rotate_x = 0
                        rotate_y = 0
                        rotate_z = 0
                        ratio = .1
                        line_width= 1
                        geometry = 1
                        topo = 1
                        color_r = 0
                        color_g = 255
                        color_b = 255
                        color_a = 255
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        # Output to Tag file
                        tagtext = "search terms related to covid-19 %: " + str(row["search"])
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1
                       
        # early indicators    
        elif j == 1:
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = 90
            translate_y = 0
            translate_z = 0
            scale_x = .3
            scale_y = scale_x
            scale_z = scale_x
            line_width= 1
            geometry = 19
            topo = 6
            color_r = 59
            color_g = 196
            color_b = 255
            color_a = 40
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            ratio = .02
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level2objectid = id
            numlevel3toroids = 1
            
            for k in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                scale_x = 1 
                scale_y = scale_x
                scale_z = scale_x
                translate_x = 30
                translate_y = 0
                translate_z = 0
                rotate_x = 90
                rotate_y = 0
                rotate_z = 0
                ratio = .02
                line_width= 1
                geometry = 7
                topo = 3
                color_r = 59
                color_g = 196
                color_b = 255
                color_a = 50
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)
                
                level3objectid = id
                numlevel4toroids = 3
                for m in range(numlevel4toroids): 
                    if m == 0:
                        id += 1
                        data = id
                        type = 5
                        parent_id = level3objectid
                        selected = 0
                        branch_level = 4
                        child_id = id
                        scale_x = row['survey_c_scalex'] 
                        scale_y = row['survey_c_scaleyz'] 
                        scale_z = scale_y
                        translate_x = 124
                        translate_y = 0
                        translate_z = 0
                        rotate_x = 0
                        rotate_y = 90
                        rotate_z = 0
                        ratio = .1
                        line_width= 1
                        geometry = 11
                        topo = 2
                        color_r = 106
                        color_g = 0
                        color_b = 149
                        color_a = 255
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        # Output to Tag file
                        tagtext = "symptoms indicate covid-19: " + str(row["survey_c"])
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1
                    elif m == 1:
                        id += 1
                        data = id
                        type = 5
                        parent_id = level3objectid
                        selected = 0
                        branch_level = 4
                        child_id = id
                        scale_x = row['survey_i_scalex'] 
                        scale_y = row['survey_i_scaleyz']
                        scale_z = scale_y
                        translate_x = 60
                        translate_y = 0
                        translate_z = 0
                        rotate_x = 0
                        rotate_y = 90
                        rotate_z = 0
                        ratio = .1
                        line_width= 1
                        geometry = 11
                        topo = 2
                        color_r = 154
                        color_g = 0
                        color_b = 101
                        color_a = 255
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        # Output to Tag file
                        tagtext = "symptoms related to influenza: " + str(row["survey_i"])
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1
                    elif m == 2:
                        id += 1
                        data = id
                        type = 5
                        parent_id = level3objectid
                        selected = 0
                        branch_level = 4
                        child_id = id
                        scale_x = row['doc_visit_scale'] 
                        scale_y = scale_x
                        scale_z = scale_x
                        translate_x = -90
                        translate_y = 0
                        translate_z = 0
                        rotate_x = 0
                        rotate_y = 0
                        rotate_z = 0
                        ratio = .1
                        line_width= 1
                        geometry = 3
                        topo = 2
                        color_r = 255
                        color_g = 168
                        color_b = 0
                        color_a = 255
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        # Output to Tag file
                        tagtext = "doctors visits related to covid: " + str(row["doc_visit"])
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1
                    
        # late indicators        
        elif j == 2:
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = 130
            translate_y = 0
            translate_z = 0
            scale_x = .78
            scale_y = scale_x
            scale_z = scale_x
            line_width= 1
            geometry = 19
            topo = 6
            color_r = 139
            color_g = 116
            color_b = 255
            color_a = 40
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            ratio = .02
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level2objectid = id
            numlevel3toroids = 4
            #parameters
            for k in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                if k == 0:
                    scale_x = row['jhu_d_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    translate_x = 0
                    geometry = 13
                    topo = 2
                    color_r = 255
                    color_g = 0
                    color_b = 0
                    rotate_x = 90
                    tagtext = "Deaths: " + str(row['jhu_d'])
                elif k == 1:
                    scale_x = row['jhu_case_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    translate_x = -33
                    geometry = 9
                    topo = 2
                    color_r = 166
                    color_g = 0
                    color_b = 89
                    rotate_x = 0
                    tagtext = "Cases: " + str(row['jhu_case'])
                elif k == 2:
                    scale_x = row['hv_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    translate_x = -121
                    geometry = 6
                    topo = 3
                    line_width = 5
                    color_r = 255
                    color_g = 168
                    color_b = 0
                    rotate_x = 0
                    tagtext = "Hospital visits: " + str(row['hv'])
                elif k == 3:
                    scale_x = row['c_tests_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    translate_x = -137
                    geometry = 16
                    topo = 5
                    color_r = 0
                    color_g = 255
                    color_b = 0
                    rotate_x = 90
                    tagtext = "Covid tests: " + str(row['c_tests'])
                color_a = 255
                translate_y = 0
                translate_z = 0
                rotate_y = 0
                rotate_z = 0
                ratio = .1
                line_width= 1
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)
                # Output to Tag file
                tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                ftag.write(tagstring)
                taginc += 1

        elif j == 3:
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = -75
            translate_y = 0
            translate_z = 0
            scale_x = .78
            scale_y = scale_x
            scale_z = scale_x
            line_width= 1
            geometry = 19
            topo = 6
            color_r = 139
            color_g = 116
            color_b = 255
            color_a = 40
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            ratio = .02
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)
            

            level2objectid = id
            numlevel3toroids = 1
            #economic indicator
            for k in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                scale_x = 1.5
                scale_y = scale_x
                scale_z = scale_x
                translate_x = 0
                translate_y = 0
                translate_z = 0
                rotate_x = 0
                rotate_y = 0
                rotate_z = 0
                ratio = .1
                line_width= 1
                geometry = 19
                topo = 4
                if row['E2_Debt/contract relief'] == 0:
                    color_r = 82
                    color_g = 90
                    color_b = 102
                elif row['E2_Debt/contract relief'] == 1:
                    color_r = 75
                    color_g = 131
                    color_b = 209
                elif row['E2_Debt/contract relief'] == 2:
                    color_r = 0
                    color_g = 94
                    color_b = 209
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)
                # Output to Tag file
                tagtext = "Debt/contract relief: " + str(row['E2_Notes'])
                tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                ftag.write(tagstring)
                taginc += 1
                
        elif j == 4:
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = -90
            translate_y = 0
            translate_z = 0
            scale_x = .78
            scale_y = scale_x
            scale_z = scale_x
            line_width= 1
            geometry = 19
            topo = 6
            color_r = 139
            color_g = 116
            color_b = 255
            color_a = 40
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            ratio = .02
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level2objectid = id
            numlevel3toroids = 1
            #cylinder objects for each reading per day
            for k in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                scale_x = 1.5
                scale_y = scale_x
                scale_z = scale_x
                translate_x = 0
                translate_y = 0
                translate_z = 0
                rotate_x = 0
                rotate_y = 0
                rotate_z = 0
                ratio = .1
                line_width= 1
                geometry = 3
                topo = 2
                if row['C6_Stay at home requirements'] == 0:
                    color_r = 82
                    color_g = 90
                    color_b = 102
                elif row['C6_Stay at home requirements'] == 1:
                    color_r = 115
                    color_g = 151
                    color_b = 209
                elif row['C6_Stay at home requirements'] == 2:
                    color_r = 75
                    color_g = 131
                    color_b = 209
                elif row['C6_Stay at home requirements'] == 3:
                    color_r = 0
                    color_g = 94
                    color_b = 209
                elif row['C6_Stay at home requirements'] == 4:
                    color_r = 157
                    color_g = 173
                    color_b = 196
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)
                # Output to Tag file
                tagtext = "Stay at home requirements: " + str(row['C6_Notes'])
                tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                ftag.write(tagstring)
                taginc += 1

        elif j == 5:
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = -105
            translate_y = 0
            translate_z = 0
            scale_x = .78
            scale_y = scale_x
            scale_z = scale_x
            line_width= 1
            geometry = 19
            topo = 6
            color_r = 139
            color_g = 116
            color_b = 255
            color_a = 40
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            ratio = .02
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level2objectid = id
            numlevel3toroids = 1
            #health policies
            for k in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                translate_x = 0
                translate_y = 0
                translate_z = 0
                rotate_x = 0
                rotate_y = 0
                rotate_z = 0
                ratio = .1
                line_width= 1
                geometry = 1
                topo = 1
                if row['H1_Public information campaigns'] == 0:
                    color_r = 82
                    color_g = 90
                    color_b = 102
                    scale_x = 1.5 
                    scale_y = scale_x
                    scale_z = scale_x
                elif row['H1_Public information campaigns'] == 1:
                    color_r = 75
                    color_g = 131
                    color_b = 209
                    scale_x = 1.5 
                    scale_y = scale_x
                    scale_z = scale_x
                elif row['H1_Public information campaigns'] == 2:
                    color_r = 0
                    color_g = 94
                    color_b = 209
                    scale_x = 1.5
                    scale_y = scale_x
                    scale_z = scale_x
                elif row['H1_Public information campaigns'] == 4:
                    color_r = 157
                    color_g = 173
                    color_b = 196
                    scale_x = 1.5
                    scale_y = scale_x
                    scale_z = scale_x
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)

                # Output to Tag file
                tagtext = "Public information campaigns: " + str(row['H1_Notes'])
                tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                ftag.write(tagstring)
                taginc += 1

if row['weekday'] == 1:
        color_r = 255
        color_g = 250
        color_b = 240
    if row['weekday'] == 2:
        color_r = 205
        color_g = 92
        color_b = 92
    if row['weekday'] == 3:
        color_r = 50
        color_g = 205
        color_b = 50
    if row['weekday'] == 4:
        color_r = 255
        color_g = 255
        color_b = 224
    if row['weekday'] == 5:
        color_r = 240
        color_g = 248
        color_b = 255
    if row['weekday'] == 6:
        color_r = 102
        color_g = 51
        color_b = 153
    if row['weekday'] == 0:
        color_r = 128
        color_g = 0
        color_b = 0